# T-matrix computation for 3D acoustic scattering 

In [2]:
import bempp.api
import numpy as np
import scipy
from numba import objmode
import numba
import math
%matplotlib inline
import matplotlib.pyplot as plt
bempp.api.show_available_platforms_and_devices()
bempp.api.set_default_device(0, 0)

0: Portable Computing Language
    0: pthread-Intel(R) Xeon(R) W-2155 CPU @ 3.30GHz


In [3]:
h = 0.03 # Size of the mesh
d = [1,0,0] # wave's travel direction
k = 5 # wavenumber

# far field points: unit sphere
unit_sphere = bempp.api.shapes.sphere(r = 1, origin=(0,0,0), h = h)
vert_sphere = unit_sphere.vertices

# scatterer: cube
dom_cube = bempp.api.shapes.sphere(r = 0.9, origin=(0,0,0), h = h)
space_cube = bempp.api.function_space(dom_cube,'P',1)

In [4]:
def Legendre_poly(q, x):  
    if(q == 0): 
        return 1 # P0 = 1 
    elif(q == 1): 
        return x # P1 = x 
    else: 
        return ((2*q - 1)*x * Legendre_poly(q-1, x)-(q-1)*Legendre_poly(q - 2, x))/q 

In [5]:
num_of_truncate_term = 20

def far_field(x):
    field = 0
    cosine_angle = x[0]/np.linalg.norm(x)
    for i in range(num_of_truncate_term):
        a = (1j/k) * (2*i+1) * spherical_bessel_function(i,[0.9,0,0]) * Legendre_poly(i,cosine_angle) / spherical_hankel_function(i,[0.9,0,0])
        field += a
    return field

In [8]:
far_list = []
for i in range(vert_sphere.shape[1]):
    far_list.append(far_field(vert_sphere[:,i]))

In [9]:
far_list[0]

(-1.4503729557995284+2.6734154577358056j)

In [10]:
eta = k

identity = bempp.api.operators.boundary.sparse.identity(space_cube, space_cube,space_cube)
slp = bempp.api.operators.boundary.helmholtz.single_layer(space_cube,space_cube,space_cube,k)
dlp = bempp.api.operators.boundary.helmholtz.double_layer(space_cube,space_cube,space_cube,k)

lhs2 = 0.5 * identity + dlp - 1j* eta *slp

In [11]:
@bempp.api.complex_callable
def exact_data(x, n, domain_index, result):
    result[0] = - np.exp(1j * k * x[0])

grid_fun = bempp.api.GridFunction(space_cube, fun=exact_data)

In [12]:
fun_exact, info_exact = bempp.api.linalg.gmres(lhs2, grid_fun, tol=1E-5)

In [13]:
from bempp.api.operators.far_field import helmholtz as helmholtz_farfield
slp_far = helmholtz_farfield.single_layer(space_cube, vert_sphere, k)
dlp_far = helmholtz_farfield.double_layer(space_cube, vert_sphere,k)
res_exact = dlp_far.evaluate(fun_exact) - 1j * eta * slp_far.evaluate(fun_exact)

In [14]:
res_exact[0,0]

(-1.4501510145114747+2.672803284794914j)

In [7]:
def normalized_spherical_harmonics(p, q, x):
    """Spherical Harmonic function of degree q"""
    azimuth = np.arctan2(x[1],x[0])
    polar = np.arccos(x[2]/np.linalg.norm(x))
    if p >= 0:
        return ((-1)**p) * scipy.special.sph_harm(p,q,azimuth,polar)
    else:
        return scipy.special.sph_harm(-p,q,azimuth,polar)*np.exp(1j*2*p*azimuth)

def spherical_bessel_function(q,x):
    """Spherical Bessel function of degree q"""
    r = np.linalg.norm(x)
    return scipy.special.spherical_jn(q,k*r)

def spherical_hankel_function(q,x):
    """Spherical hankel function of degree q"""
    r = np.linalg.norm(x)
    return scipy.special.spherical_jn(q, k*r) + 1j * scipy.special.spherical_yn(q, k*r)

def regular_spherical_wavefunctions(p,q,x):
    """Regular Spherical Wavefunction"""
    return spherical_bessel_function(q,x)*normalized_spherical_harmonics(p,q,x)


In [ ]:
deg = 10
sca = 0 
for q in range(deg+1):
    for p in range(-q,q+1):
        coeff = 4*np.pi*((1j)**q)*normalized_spherical_harmonics(-p,q,d)/(-1)**(abs(p))

        @bempp.api.complex_callable
        def rhs_data(x, n, domain_index, result):
            with objmode():
                result[0] =  -  regular_spherical_wavefunctions(p,q,x)

        rhs_fun = bempp.api.GridFunction(space_cube, fun = rhs_data)
        fun_appro, info_appro = bempp.api.linalg.gmres(lhs2, rhs_fun, tol=1E-5)
        res_appro = dlp_far.evaluate(fun_appro) - 1j * eta * slp_far.evaluate(fun_appro)
        sca += res_appro*coeff

In [ ]:
sca[0,0]